# Exercise 5: Decision Trees

# In-Class Basics


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats # We will use stats.entropy to calculate the entropy of a distribution

To implement decision trees, we will need recursive functions. Here is a simple example to calculate the factorial of an integer (n!). Make sure you understand how this works before moving on.

In [2]:
def factorial(n):
    
    if n==1:
        return 1
    else:
        return n*factorial(n-1)

In [3]:
factorial(5)

120

Load and prepare the data

In [4]:
f = np.load("Ex5_data.npz") # Make sure to also download this file from github
X = f["arr_0"]
y = f["arr_1"]

# We have a total of 15k examples with 16 features per example. 
# All features are binary - so either True (1) or False (0).
# Additionally we have a vector of class labels y. 
# For each example the class is either 0 or 1.
# The features contain enough information to (partially) infer the class.
print(X.shape,y.shape)
print(X[0],y[0])

# Split into training (10k examples) and testing (5k examples) data
X_train = X[:10000]
X_test = X[10000:]

y_train = y[:10000]
y_test = y[10000:]



((15000, 16), (15000,))
(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False]), 0)


In [5]:
# Let's define a very simple format for specifying decistion trees: 
# nested python lists
# Each node is specified by 3 quantities:
# - what feature to split on (an integer specifying the column of the data matrix)
# - what to do if the feature == True 
#     This can be either another list (tree) specifying a further subdivision or an integer specifying the class
# - what to do if the feature == False 
#     (again, either a list or an integer)

# Simple example:
tree_1 = [5, 0, 1] 
# This should be interpreted as:
# if x[5] == True choose class 0,
# else choose class 1

# Slightly more complicated
tree_2 = [5,1, [3,0,1]] 
# This should be interpreted as:
# if x[5] == True choose class 1
# else
#   if x[3] == True choose class 0 
#   else choose class 1

In [6]:
# Recursive function to apply a decision tree to a given exampke x

def apply_decision_tree(x, tree):
        
    # As specified above, a tree can either be a list or a number
    # If it is a list, we have to go deeper, 
    # if it is a number, we return it.
        
    # Test if the object is a list
    if tree.__class__ == list:
        
        # If it is a list then we interprete it as:
        # [feature to test, tree to follow if True, tree to follow if False]
        
        # Test if the feature is true
        if x[tree[0]]:
            # go deeper on the feature==True path
            return apply_decision_tree(x,tree[1])
        else:
            # go deeper on the feature==False path
            return apply_decision_tree(x,tree[2])
        
    else:
        # If the 'tree' is just a number, we have reached a leave node. 
        # Just return the number
        return tree
        


# Homework Problems

 * Write a function `calc_gain(X,y,feature)` that takes a matrix of training events `X`, correpsponding labels `y` and a feature index `feature` and return the entropy gain (as defined in the lecture) in y when splitting according to feature. You can find a skeleton version of the function below.
 * Write a function `build_decision_tree(X,y,features)` that takes a matrix of training events `X`, correpsponding labels `y` and a list of `features` and returns a decision tree using the format specified above. You can find a skeleton version of the function below.
 * Use `build_decision_tree` to calculate a decision tree for the X_test sample. Apply it to all examples in `X_test` to obtain a vector of predictions `y_pred`. Compare it to `y_test` and calculate the accuracy (the fraction of examples where the prediction and true label are identical). The code for doing this is already available below and simply needs to be run once `build_decision_tree` and `calc_gain` are finished.

In [7]:
def calc_gain(X,y,feature):

    # This gives a list of True/False that can be used to select exampkles where the feature is true
    # the logical inversion is given by ~sel
    sel = (X[:,feature]==True)
    
    # First calculate the entropy for y as well as 
    # for the subsets of y that pass and fail the selection, respectively
    
    # -> Your code goes here
    
    # Then calculate the fraction of examples that pass and fail the selection and use it to 
    # calculate the average entropy after the selection
    
    # -> Your code goes here
    
    # Finally, return the difference in entropy of y and the average entropy you calculated above
    
    # -> Your code goes here

In [8]:
def build_decision_tree(X,y, features):

    # Check if all the examples in y are from the same class. 
    # In that case, return this class.

    # -> Your code goes here

    # Check if we have any features left to try (ie if len(features) is bigger than zero)
    # If not, return the class that has the larger fraction of examples in y 
    # (at this point in the selection chain)
    
    # -> Your code goes here

    # Find the feature that yields the largest information gain (call this best_feature)
    # You can use the calc_gain function we defined above and test all available features
    
    # -> Your code goes here

    # Build new matrices of examples (X_pass and y_pass) for the 
    # examples where the selected feature is True as well as 
    # X_fail and y_fail for those where the feature is False
    # Also build a copy of the list of features without the one we select on         
    new_features = [i for i in features if not i==best_feature]
    
    sel = X[:,best_feature] == True

    X_pass = X[sel]
    y_pass = y[sel]

    X_fail = X[~sel]
    y_fail = y[~sel]

    # Check if one of the new subcategories is empty. 
    # If it is, we set the output class for this subcategory
    # to the predominant class (at the current point in the selection)
    # Otherwise, call build_decision_tree on it (remember to use new_features!)
    
    # -> Your code goes here

    # Build and return a list with three entries: 
    # [best_feature, 
    #  decision (either class or further build_decision_tree call for pass),
    #  decision (either class or further build_decision_tree call for fail)]

    # -> Your code goes here

In [ ]:
# The code below can be used to build the tree and calculate the accuracy 
# once calc_gain and build_precision_tree are properly implemented

tree = build_decision_tree(X_train,y_train,range(X_train.shape[1]))

# Calculate predictions on the test data
y_pred = np.array([apply_decision_tree(X_test[i],tree) for i in range(X_test.shape[0])])

# Calculate accuracy
1.*sum(y_pred == y_test) / y_pred.shape[0]